### Audiobooks business case

In [1]:
import numpy as np
from sklearn import preprocessing 

In [2]:
raw_data = np.loadtxt('Audiobooks_data.csv',delimiter=',')
unscaled_inputs_all = raw_data[:, 1:-1]
targets_all = raw_data[:, -1]

### Shuffle the dataset

In [3]:
shuffled_indices = np.arange(unscaled_inputs_all.shape[0])
np.random.shuffle(shuffled_indices)

unscaled_inputs_all = unscaled_inputs_all[shuffled_indices]
targets_all = targets_all[shuffled_indices]

### Balance the dataset

In [4]:
num_one_targets = int(np.sum(targets_all))
zero_targets_counter = 0
indices_to_remove = []

for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)
            
            
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)        

### Standardize the inputs

In [5]:
scaled_inputs = preprocessing.scale(unscaled_inputs_equal_priors)

### Shuffle the dataset 

In [6]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]
shuffled_inputs.shape

(4474, 10)

### Split the dataset into train, validation and test

In [7]:
samples_count = shuffled_inputs.shape[0]

train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1795.0 3579 0.5015367421067337
225.0 447 0.5033557046979866
217.0 448 0.484375


### Save the three datasets in *.npz 

In [8]:
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Model

In [10]:
import tensorflow as tf

In [22]:
input_size = 10
output_size = 2
hidden_layer_size = 50

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
    tf.keras.layers.Dense(output_size, activation = 'softmax')
])

model.compile(optimizer='adam', loss = 'sparse_categorical_crossentropy', metrics=['accuracy'])

In [26]:
batch_size = 100
max_epochs =100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(
    train_inputs,
    train_targets,
    batch_size = batch_size,
    epochs = max_epochs,
    callbacks=[early_stopping],
    validation_data = (validation_inputs, validation_targets),
    verbose = 2
)

Epoch 1/100
36/36 - 0s - loss: 0.3342 - accuracy: 0.8220 - val_loss: 0.3458 - val_accuracy: 0.8233
Epoch 2/100
36/36 - 0s - loss: 0.3328 - accuracy: 0.8226 - val_loss: 0.3416 - val_accuracy: 0.8121
Epoch 3/100
36/36 - 0s - loss: 0.3346 - accuracy: 0.8175 - val_loss: 0.3531 - val_accuracy: 0.8166
Epoch 4/100
36/36 - 0s - loss: 0.3362 - accuracy: 0.8164 - val_loss: 0.3424 - val_accuracy: 0.8300


### Test the model

In [27]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 3ms/step - loss: 0.3155 - accuracy: 0.8527
